<h1> <center> Amazon Fake Reviews </center> </h1>

In [1]:
#!pip install langdetect
#!pip install language_check

In [2]:
#General purpose
import os
#Data handling
import pandas as pd
import numpy as np
import datetime
#Data Visualization
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline 
#Modelling 
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
from langdetect import detect #to filter non-english reviews.
import language_check #to detect errors
import sklearn
from sklearn.feature_extraction.text import CountVectorizer

In [4]:
## nltk downloads
# nltk.download('punkt') # At first you have to download these nltk packages.
# nltk.download('stopwords')
# nltk.download('wordnet')

In [3]:
#Load dataset
df=pd.read_csv('../data/exports/test_appliances.csv')
df.drop('Unnamed: 0',axis=1, inplace=True)
df.head()

,asin,review_date,reviewerID,reviewerName,summary,overall,reviewText,vote,date,title,description,image,main_cat,brand,rank,product_year
0,B00009W3I4,2013-08-22,A34A1UP40713F8,James. Backus,Great product,5,I like this as a vent as well as something tha...,0,2003-05-14,HEARTLAND 21000 Energy Saving Dryer Vent Closure,Replace your old vent with the Heartland Natur...,1.0,Tools & Home Improvement,HEARTLAND,5633.0,2003.0
1,B00009W3PA,2016-02-08,A1AHW6I678O6F2,kevin.,Five Stars,5,good item,0,2006-05-26,Certified Appliance Accessories 3-Wire Closed-...,Some people are surprised to learn that a new ...,1.0,Tools & Home Improvement,Certified Appliance Accessories,135619.0,2006.0
2,B00009W3PA,2015-08-05,A8R48NKTGCJDQ,CDBrannom,Five Stars,5,Fit my new LG dryer perfectly.,0,2006-05-26,Certified Appliance Accessories 3-Wire Closed-...,Some people are surprised to learn that a new ...,1.0,Tools & Home Improvement,Certified Appliance Accessories,135619.0,2006.0
3,B00009W3PA,2015-04-24,AR3OHHHW01A8E,Calvin E Reames,Perfect size,5,Good value for electric dryers,0,2006-05-26,Certified Appliance Accessories 3-Wire Closed-...,Some people are surprised to learn that a new ...,1.0,Tools & Home Improvement,Certified Appliance Accessories,135619.0,2006.0
4,B00009W3PA,2015-03-21,A2CIEGHZ7L1WWR,albert j. kong,Five Stars,5,Price and delivery was excellent.,0,2006-05-26,Certified Appliance Accessories 3-Wire Closed-...,Some people are surprised to learn that a new ...,1.0,Tools & Home Improvement,Certified Appliance Accessories,135619.0,2006.0
